# 1. Import Dependencies

In [1]:
!pip install stable-baselines3[extra]
import gym 
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
using gpu 4


# 2. Load Environment

In [2]:
environment_name = "CartPole-v0"

In [3]:
env = gym.make(environment_name)

In [4]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    while not done:
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:42.0
Episode:2 Score:19.0
Episode:3 Score:8.0
Episode:4 Score:20.0
Episode:5 Score:14.0


# 3. Understanding The Environment


In [5]:
# 0-push cart to left, 1-push cart to the right
env.action_space.sample()


0

In [6]:
# [cart position, cart velocity, pole angle, pole angular velocity]
env.observation_space.sample()

array([ 3.4063570e+00, -3.0459490e+38, -1.8623586e-01,  3.2697114e+38],
      dtype=float32)

# 3. Train an RL Model

In [7]:
log_path = os.path.join('Training','Logs')

In [8]:
log_path

'Training/Logs'

In [9]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose = 1)

Using cuda device


In [10]:
model.learn(total_timesteps=20000)

-----------------------------
| time/              |      |
|    fps             | 456  |
|    iterations      | 1    |
|    time_elapsed    | 4    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 389         |
|    iterations           | 2           |
|    time_elapsed         | 10          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.007930074 |
|    clip_fraction        | 0.0947      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | 0.0197      |
|    learning_rate        | 0.0003      |
|    loss                 | 5.42        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0139     |
|    value_loss           | 53.6        |
-----------------------------------------
----------------------------------

# 4. Save and Reload Model

In [17]:
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_model')

In [18]:
model.save(PPO_path)

In [ ]:
del model

In [44]:
model = PPO.load('PPO_model', env=env)

FileNotFoundError: [Errno 2] No such file or directory: 'PPO_model.zip'

# 5. Evaluation


In [19]:
evaluate_policy(model, env, n_eval_episodes=10,)

(200.0, 0.0)

In [20]:
env.close()

# 6. Test Model

In [21]:
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    if done: 
        print('info', info)
        break

info [{'terminal_observation': array([-2.4028096 , -1.8650149 , -0.04216713,  0.22754177], dtype=float32)}]


# 7. Viewing Logs in Tensorboard

In [22]:
training_log_path = os.path.join(log_path, 'PPO_3')

# 8. Adding a callback to the training Stage

In [23]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
import os
save_path = os.path.join('Training', 'Saved Models')
log_path = os.path.join('Training', 'Logs')
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=190, verbose=1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=10000, 
                             best_model_save_path=save_path, 
                             verbose=1)
model = PPO('MlpPolicy', env, verbose = 1,)
model.learn(total_timesteps=20000, callback=eval_callback)
model_path = os.path.join('Training', 'Saved Models', 'best_model')
model = PPO.load(model_path, env=env)
evaluate_policy(model, env, n_eval_episodes=10,)
env.close()

Using cuda device
-----------------------------
| time/              |      |
|    fps             | 782  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 589         |
|    iterations           | 2           |
|    time_elapsed         | 6           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008944316 |
|    clip_fraction        | 0.103       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.00321    |
|    learning_rate        | 0.0003      |
|    loss                 | 6.12        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0159     |
|    value_loss           | 46.3        |
-----------------------------------------
----------------

# 9. Changing Policies


In [ ]:
net_arch=[dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]
model = PPO('MlpPolicy', env, verbose = 1, policy_kwargs={'net_arch': net_arch})
model.learn(total_timesteps=20000, callback=eval_callback)

# 10. Using an Alternate Algorithm

In [ ]:
from stable_baselines3 import DQN
model = DQN('MlpPolicy', env, verbose = 1,)
model.learn(total_timesteps=20000, callback=eval_callback)
dqn_path = os.path.join('Training', 'Saved Models', 'DQN_model')
model.save(dqn_path)
model = DQN.load(dqn_path, env=env)
evaluate_policy(model, env, n_eval_episodes=10,)
env.close()